In [ ]:
from langchain_chroma import Chroma
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_tavily import TavilySearch
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, Literal, Optional
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [32]:
load_dotenv()
model = ChatGroq(
    model=os.getenv("model"),
    api_key=os.getenv("api_key")
)

embedding_model = OllamaEmbeddings(model="llama3.2")

In [33]:
loader = PyPDFLoader(file_path="Attention.pdf")
docs = loader.load()
print(len(docs))

11


In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 80)
chunks = text_splitter.split_documents(docs)

print(len(chunks))

234


In [35]:
vector_store = Chroma.from_documents(
    embedding=embedding_model,
    documents=chunks,
    collection_name= "my_collection"
)

In [36]:
retrievers = vector_store.as_retriever(search_type = "mmr", search_kwargs = {'k' : 3, 'lambda_mult' : 0.7})


In [37]:
def page_content_ext(chunks : list[Document]):
    pass